# pycarta MQTT Messaging

This notebook demonstrates MQTT messaging capabilities including publishing, subscribing, and async operations.

## Prerequisites

- Valid Carta authentication (see `01_authentication.ipynb`)
- Understanding of MQTT protocol basics
- MQTT broker access (configured through Carta)

## Setup

In [2]:
import pycarta as pc
from pycarta.mqtt import publish, subscribe
import asyncio
import json
from datetime import datetime

# Ensure you're authenticated
pc.login(interactive=True)  # Uncomment and authenticate as needed

print("MQTT setup complete - ensure authentication before running examples")

MQTT setup complete - ensure authentication before running examples


## Publishing Messages

Use the `@publish` decorator to automatically publish function results:

In [3]:
# Simple temperature sensor example
@publish("sensors/temperature")
def read_temperature():
    """Simulate reading a temperature sensor."""
    import random
    temperature = round(random.uniform(20.0, 30.0), 1)
    return {
        "temperature": temperature,
        "unit": "celsius",
        "timestamp": datetime.now().isoformat(),
        "sensor_id": "temp_001"
    }

print("Temperature sensor publishing function defined")
print("Call read_temperature() to publish a temperature reading")

Temperature sensor publishing function defined
Call read_temperature() to publish a temperature reading


In [4]:
# Test the temperature publishing
try:
    # result = read_temperature()  # Uncomment when MQTT is configured
    # print(f"Published temperature reading: {result}")
    print("Temperature publishing test ready (uncomment when MQTT configured)")
except Exception as e:
    print(f"Publishing error: {e}")
    print("Note: Requires MQTT broker configuration")

Temperature publishing test ready (uncomment when MQTT configured)


## Multiple Sensor Publishing

In [ ]:
# Humidity sensor
@publish("sensors/humidity")
def read_humidity():
    """Simulate reading a humidity sensor."""
    import random
    humidity = round(random.uniform(40.0, 80.0), 1)
    return {
        "humidity": humidity,
        "unit": "percent",
        "timestamp": datetime.now().isoformat(),
        "sensor_id": "hum_001"
    }

# Pressure sensor
@publish("sensors/pressure")
def read_pressure():
    """Simulate reading a pressure sensor."""
    import random
    pressure = round(random.uniform(1000.0, 1030.0), 1)
    return {
        "pressure": pressure,
        "unit": "hPa",
        "timestamp": datetime.now().isoformat(),
        "sensor_id": "pres_001"
    }

# System status
@publish("system/status")
def publish_system_status():
    """Publish system status information."""
    import psutil
    return {
        "cpu_percent": psutil.cpu_percent(),
        "memory_percent": psutil.virtual_memory().percent,
        "timestamp": datetime.now().isoformat(),
        "status": "healthy"
    }

# Asynchronous operation
@publish("system/check")
async def async_system_checker():
    return {
        "status": "RUNNING",
        "timestamp": datetime.now().isoformat(),
    }

print("Multiple sensor publishing functions defined")
print("Topics: sensors/humidity, sensors/pressure, system/status")

Multiple sensor publishing functions defined
Topics: sensors/humidity, sensors/pressure, system/status


## Subscribing to Messages

Use the `@subscribe` decorator to handle incoming messages:

In [7]:
# System alert handler
@subscribe("alerts/system")
def handle_system_alert(message):
    """Handle system alerts from MQTT."""
    print(f"🚨 System Alert Received: {message}")
    
    # Process the alert
    if isinstance(message, dict):
        alert_type = message.get('type', 'unknown')
        severity = message.get('severity', 'info')
        description = message.get('description', 'No description')
        
        print(f"   Type: {alert_type}")
        print(f"   Severity: {severity}")
        print(f"   Description: {description}")
        
        # Take action based on severity
        if severity == 'critical':
            print("   🔴 CRITICAL ALERT - Immediate action required!")
        elif severity == 'warning':
            print("   🟡 WARNING - Monitor situation")
        else:
            print("   🟢 INFO - Logged for reference")

# Sensor data handler
@subscribe("sensors/+")  # Wildcard to match all sensor topics
def handle_sensor_data(message):
    """Handle sensor data from all sensors."""
    print(f"📊 Sensor Data: {message}")
    
    if isinstance(message, dict):
        sensor_id = message.get('sensor_id', 'unknown')
        timestamp = message.get('timestamp', 'unknown')
        print(f"   Sensor: {sensor_id} at {timestamp}")

# Command handler
@subscribe("commands/device")
def handle_device_command(message):
    """Handle device commands."""
    print(f"⚙️ Device Command: {message}")
    
    if isinstance(message, dict):
        command = message.get('command', 'unknown')
        device_id = message.get('device_id', 'unknown')
        
        print(f"   Command: {command} for device {device_id}")
        
        # Simulate command execution
        if command == 'restart':
            print(f"   Restarting device {device_id}...")
        elif command == 'shutdown':
            print(f"   Shutting down device {device_id}...")
        elif command == 'status':
            print(f"   Checking status of device {device_id}...")
        else:
            print(f"   Unknown command: {command}")

# Asynchronous subscriber
@subscribe("system/check")
async def async_system_health_check(content):
    print("Health Check:", str(content))

print("MQTT subscribers defined")
print("Subscribed to: alerts/system, sensors/+, commands/device")

MQTT subscribers defined
Subscribed to: alerts/system, sensors/+, commands/device


## MQTT Quality of Service (QoS)

Different levels of message delivery guarantee:

In [ ]:
# QoS examples
@publish("critical/alerts", qos=2)  # Exactly once delivery
def publish_critical_alert(alert_data):
    """Publish critical alerts with highest QoS."""
    return {
        "alert_id": alert_data.get("id"),
        "type": "critical",
        "message": alert_data.get("message"),
        "timestamp": datetime.now().isoformat(),
        "requires_acknowledgment": True
    }

@publish("monitoring/metrics", qos=1)  # At least once delivery
def publish_monitoring_data(metrics):
    """Publish monitoring metrics with medium QoS."""
    return {
        "metrics": metrics,
        "timestamp": datetime.now().isoformat(),
        "source": "monitoring_service"
    }

@publish("logs/debug", qos=0)  # Fire and forget
def publish_debug_log(log_message):
    """Publish debug logs with lowest QoS."""
    return {
        "level": "debug",
        "message": log_message,
        "timestamp": datetime.now().isoformat()
    }

print("QoS examples defined:")
print("  QoS 0: Fire and forget (debug logs)")
print("  QoS 1: At least once delivery (monitoring)")
print("  QoS 2: Exactly once delivery (critical alerts)")

## TLS/SSL Security

Secure MQTT connections with credentials:

In [8]:
# Example of secure MQTT configuration
def setup_secure_mqtt():
    """Example of setting up secure MQTT with TLS/SSL."""
    print("""
    SECURE MQTT SETUP:
    
    For secure MQTT connections, you need:
    
    1. CA Certificate (ca_cert_path)
    2. Client Certificate (client_cert_path) 
    3. Private Key (client_key_path)
    
    Configuration example:
    
    from pycarta.mqtt import MQTTClient
    
    client = MQTTClient(
        host="secure-mqtt-broker.com",
        port=8883,  # Standard secure MQTT port
        ca_cert_path="/path/to/ca.crt",
        client_cert_path="/path/to/client.crt",
        client_key_path="/path/to/client.key",
        tls=True
    )
    
    Credentials can be managed through Carta secrets:
    
    from pycarta.admin.secret import put_secret, get_secret
    
    # Store certificate paths
    put_secret("mqtt_ca_cert", "/path/to/ca.crt")
    put_secret("mqtt_client_cert", "/path/to/client.crt")
    put_secret("mqtt_client_key", "/path/to/client.key")
    
    # Retrieve for use
    ca_cert = get_secret("mqtt_ca_cert")
    client_cert = get_secret("mqtt_client_cert")
    client_key = get_secret("mqtt_client_key")
    """)

setup_secure_mqtt()


    SECURE MQTT SETUP:

    For secure MQTT connections, you need:

    1. CA Certificate (ca_cert_path)
    2. Client Certificate (client_cert_path) 
    3. Private Key (client_key_path)

    Configuration example:

    from pycarta.mqtt import MQTTClient

    client = MQTTClient(
        host="secure-mqtt-broker.com",
        port=8883,  # Standard secure MQTT port
        ca_cert_path="/path/to/ca.crt",
        client_cert_path="/path/to/client.crt",
        client_key_path="/path/to/client.key",
        tls=True
    )

    Credentials can be managed through Carta secrets:

    from pycarta.admin.secret import put_secret, get_secret

    # Store certificate paths
    put_secret("mqtt_ca_cert", "/path/to/ca.crt")
    put_secret("mqtt_client_cert", "/path/to/client.crt")
    put_secret("mqtt_client_key", "/path/to/client.key")

    # Retrieve for use
    ca_cert = get_secret("mqtt_ca_cert")
    client_cert = get_secret("mqtt_client_cert")
    client_key = get_secret("mqtt_client_key"

## Real-World Example: IoT Data Pipeline

Complete example showing sensor data collection and processing:

In [ ]:
import json
from datetime import datetime, timedelta
import random

class IoTDataPipeline:
    """Example IoT data pipeline using MQTT."""
    
    def __init__(self):
        self.sensor_data_buffer = []
        self.alert_threshold = {
            'temperature': {'min': 18, 'max': 32},
            'humidity': {'min': 30, 'max': 85},
            'pressure': {'min': 995, 'max': 1035}
        }
    
    @publish("iot/sensors/temperature")
    def collect_temperature(self, sensor_id="temp_sensor_01"):
        """Collect temperature data from sensor."""
        temp = round(random.uniform(15, 35), 2)
        data = {
            'sensor_id': sensor_id,
            'type': 'temperature',
            'value': temp,
            'unit': 'celsius',
            'timestamp': datetime.now().isoformat(),
            'location': 'Building A, Floor 2'
        }
        
        # Check for alerts
        if temp < self.alert_threshold['temperature']['min'] or temp > self.alert_threshold['temperature']['max']:
            self.send_alert('temperature', temp, sensor_id)
        
        return data
    
    @publish("iot/sensors/humidity")
    def collect_humidity(self, sensor_id="hum_sensor_01"):
        """Collect humidity data from sensor."""
        humidity = round(random.uniform(25, 90), 2)
        data = {
            'sensor_id': sensor_id,
            'type': 'humidity',
            'value': humidity,
            'unit': 'percent',
            'timestamp': datetime.now().isoformat(),
            'location': 'Building A, Floor 2'
        }
        
        # Check for alerts
        if humidity < self.alert_threshold['humidity']['min'] or humidity > self.alert_threshold['humidity']['max']:
            self.send_alert('humidity', humidity, sensor_id)
        
        return data
    
    @publish("iot/alerts/environmental")
    def send_alert(self, sensor_type, value, sensor_id):
        """Send environmental alerts."""
        threshold = self.alert_threshold[sensor_type]
        
        if value < threshold['min']:
            severity = 'warning'
            message = f"{sensor_type.title()} too low"
        elif value > threshold['max']:
            severity = 'critical' if value > threshold['max'] * 1.1 else 'warning'
            message = f"{sensor_type.title()} too high"
        
        alert = {
            'alert_id': f"alert_{datetime.now().strftime('%Y%m%d_%H%M%S')}",
            'sensor_id': sensor_id,
            'sensor_type': sensor_type,
            'current_value': value,
            'threshold_min': threshold['min'],
            'threshold_max': threshold['max'],
            'severity': severity,
            'message': message,
            'timestamp': datetime.now().isoformat(),
            'location': 'Building A, Floor 2'
        }
        
        print(f"🚨 Environmental Alert: {message} (Value: {value})")
        return alert
    
    @subscribe("iot/alerts/+")
    def handle_alerts(self, message):
        """Handle all IoT alerts."""
        if isinstance(message, dict):
            severity = message.get('severity', 'info')
            alert_message = message.get('message', 'Unknown alert')
            sensor_id = message.get('sensor_id', 'Unknown sensor')
            
            print(f"Alert Handler - {severity.upper()}: {alert_message} from {sensor_id}")
            
            # Take automated action for critical alerts
            if severity == 'critical':
                print("  → Triggering automated response")
                print("  → Notifying maintenance team")
                print("  → Logging to incident management system")
    
    @subscribe("iot/commands/sensors")
    def handle_sensor_commands(self, message):
        """Handle sensor control commands."""
        if isinstance(message, dict):
            command = message.get('command', 'unknown')
            sensor_id = message.get('sensor_id', 'all')
            
            print(f"Sensor Command: {command} for {sensor_id}")
            
            if command == 'calibrate':
                print(f"  → Calibrating sensor {sensor_id}")
            elif command == 'reset':
                print(f"  → Resetting sensor {sensor_id}")
            elif command == 'status':
                print(f"  → Checking status of sensor {sensor_id}")
            else:
                print(f"  → Unknown command: {command}")
    
    def simulate_data_collection(self, duration_minutes=1):
        """Simulate continuous data collection."""
        print(f"Starting data collection simulation for {duration_minutes} minute(s)...")
        
        end_time = datetime.now() + timedelta(minutes=duration_minutes)
        
        while datetime.now() < end_time:
            try:
                # Collect data from all sensors
                temp_data = self.collect_temperature()
                humidity_data = self.collect_humidity()
                
                print(f"Collected: T={temp_data['value']}°C, H={humidity_data['value']}%")
                
                # Wait before next collection
                import time
                time.sleep(10)  # Collect every 10 seconds
                
            except KeyboardInterrupt:
                print("Data collection stopped by user")
                break
            except Exception as e:
                print(f"Error in data collection: {e}")
        
        print("Data collection simulation complete")

# Create IoT pipeline instance
iot_pipeline = IoTDataPipeline()

print("IoT Data Pipeline created")
print("Use iot_pipeline.simulate_data_collection() to start simulation")
print("Sensors: temperature, humidity with automated alerting")

## Testing the IoT Pipeline

In [ ]:
# Test individual sensor readings
try:
    # Test temperature reading
    print("Testing temperature sensor:")
    temp_reading = iot_pipeline.collect_temperature()
    print(f"Temperature: {temp_reading['value']}°C")
    
    # Test humidity reading
    print("\nTesting humidity sensor:")
    humidity_reading = iot_pipeline.collect_humidity()
    print(f"Humidity: {humidity_reading['value']}%")
    
    # Simulate an alert by forcing extreme values
    print("\nSimulating high temperature alert:")
    alert = iot_pipeline.send_alert('temperature', 40.0, 'test_sensor')
    print(f"Alert generated: {alert['message']}")
    
except Exception as e:
    print(f"Testing error: {e}")
    print("Note: Requires MQTT broker configuration for full functionality")

## MQTT Best Practices

In [ ]:
print("""
MQTT BEST PRACTICES:

1. Topic Design:
   - Use hierarchical topics: "sensors/building1/floor2/temperature"
   - Avoid spaces and special characters
   - Use consistent naming conventions
   - Consider using device/sensor IDs in topics

2. Message Format:
   - Use JSON for structured data
   - Include timestamps in all messages
   - Add metadata (sensor_id, location, units)
   - Keep messages reasonably sized (< 256KB)

3. Quality of Service (QoS):
   - QoS 0: Debug logs, non-critical telemetry
   - QoS 1: Important metrics, monitoring data
   - QoS 2: Critical alerts, commands, control messages

4. Security:
   - Always use TLS/SSL in production
   - Implement proper authentication
   - Use client certificates for device auth
   - Regular credential rotation

5. Error Handling:
   - Implement retry logic for failed publishes
   - Handle disconnections gracefully
   - Log connection issues
   - Use appropriate timeouts

6. Performance:
   - Use async operations for high-throughput
   - Batch messages when possible
   - Monitor broker load and client connections
   - Implement appropriate buffering

7. Monitoring:
   - Track message delivery rates
   - Monitor connection health
   - Alert on failed publishes/subscriptions
   - Log message processing times
""")

## Next Steps

After setting up MQTT messaging:
1. Integrate with data management for persistence (see `05_data_management.ipynb`)
2. Connect with Seven Bridges for data processing (see `06_seven_bridges.ipynb`)
3. Use services to create MQTT management APIs (see `03_services.ipynb`)
4. Implement monitoring and alerting systems
5. Scale for production workloads